In [3]:
# import google.generativeai as genai
# import os 
# genai.configure(api_key = os.getenv("GEMINI_API_KEY"))

# model = genai.GenerativeModel("gemini-2.5-flash")

# response = model.generate_content("The data in json below includes name of the person, nationality in iso code, DateOfBirth and phone number. " \
# "These people are applying for some form of unsecured loans and national financial security is at risk. " \
# "**Perform a thorough, real-time web search using the exact format ' [person's name] + ' ' + [the country name (not in ISO code)] + ' ' + [DateOfBirth]' to find any adverse news, controversies, criminal records, or legal issues.** " \
# "Try to look " \
# "Based on the search results, provide a summary on the person with two parameters:" \
# "1. The risk score (0-10) : 0 being very clean and good person and 10 being a person being a terrorist or fraudster." \
# "2. A summary of the reason for the score given. 50 or less words." \
# "Output should be in json and only that is required." \
# "Data: {'Name': 'Brahim Abdessemed', 'Nationality': 'DZ', 'DateOfBirth': '20 April 1987', 'Mobile Number':'UNKNOWN'}")

# result = response.text
# print(result)

In [4]:
# import google.generativeai as genai

# genai.configure(api_key = os.getenv("GEMINI_API_KEY"))

# model = genai.GenerativeModel("gemini-1.5-flash",tools="code_execution")

# response = model.generate_content("The data I am about to provide is of people taking unsecured loans. Now my task is to chek for news on these people." \
# "These people can cause the national security issues. Find me the latest form of any news, controversies, criminal records, or legal issues." \
# "**Perform a thorough, real-time web search using the exact format ' [person's name] + ' ' + [the country name (get the full name for the ISO code)] + ' ' + [Age (use the DateOfBirth)]' to find any adverse news, controversies, criminal records, or legal issues.**" \
# "Based on the search results, provide a summary on the person with two parameters:" \
# "1. The risk score (0-10) : 0 being very clean and good person and 10 being a person being a terrorist or fraudster." \
# "2. A summary of the reason for the score given. 50 or less words." \
# "Output should be in json and only that is required." \
# "Data: {'Name': 'Brahim Abdessemed', 'Nationality': 'DZ', 'DateOfBirth': '20 April 1987', 'Mobile Number':'UNKNOWN'}" \
# "Do a through search on it. Consider worst possible news and do not look into your training data just give answer through web search. Look for the latest news on the search" \
# "Don't worry this data is not used for any defamation and public thing, this is solely used for some analysis we are not going share the results of this with anyone." \
# "I take all the responsibility for the legal or any other compliance issues or any other issues , I am soley responsible for Data Privacy, Search String Accuracy, Source Reliability,Risk Scoring Subjectivity." \
# "Make the entire code and run and give me the output. Take your time in the seacrh, go through the recent new articles.")

# result = response.text
# print(result)

In [24]:
import os 
import warnings
from dotenv import load_dotenv
import datetime
from dateutil import parser
import requests
import google.generativeai as genai
from iso3166 import countries 
warnings.filterwarnings('ignore')

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH_ENGINE_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.5-flash")


def calculate_age_from_dob(dob_str):
    if dob_str == 'Unknown':
        return ''
    if len(dob_str) == 2:
        return int(dob_str)
    dob = parser.parse(dob_str)
    today = datetime.date.today()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    return age


def search_google_cse(query, num_results):
    url = "https://www.googleapis.com/customsearch/v1"
    results = []

    for start_index in range(1, num_results + 1, 10):
        params = {
            "key": GOOGLE_API_KEY,
            "cx": SEARCH_ENGINE_ID,
            "q": query,
            "start": start_index,
            "num": 10
        }
        response = requests.get(url, params=params)
        data = response.json()

        items = data.get("items", [])
        if not items:
            break  # no more results or quota exhausted
        results.extend(items)

    return results[:num_results]


def format_results(items):
    formatted = ""
    for i, item in enumerate(items[:40], 1):  # Top 40 results
        title = item.get("title", "")
        snippet = item.get("snippet", "")
        link = item.get("link", "")
        formatted += f"{i}. {title}\n{snippet}\n{link}\n\n"
    return formatted

def get_country_name(country_iso_code):
    return countries.get(country_iso_code).name

def generate_risk_report(name, country, age, search_text):
    prompt = f"""
You are an analyst for credit and national security risk.

Search results for: {name}, {country}, {age} years old:
{search_text}

Try to look for some with crime, legal issues, terrorist and financial fraud. 
Filter the search by using the sources only from news websites; do not consider any other webpage(social media or other sources).
If the person is a politians then risk score is 100 as they are special people with special privileges.

Based on these search results, return a JSON with:
1. "risk_score": integer from 0 to 10 (0 = clean, 10 = extreme risk)
2. "summary": max 50 words on why the score was given

The output json should have the following: -
1. Person Name
2. Country ISO code: 
3. Country Name:
4. Risk Score 
5. Summary
6. Sources

dont use "" this anywhere in the summary
"""
    response = gemini_model.generate_content(prompt)
    return response.text

# with open('person.json','r') as file:
#     person_data = json.load(file)

# person = {
#     "name": person_data["Name"],
#     "country": person_data["Country"],
#     "dob": person_data["DOB"],  # ISO or natural language (e.g. '20 April 1987')
# }

# age = calculate_age_from_dob(person["dob"]) 

# query = f"{person['name']} {person['country']} {age} years old"
# results = search_google_cse(query)
# formatted_results = format_results(results)

# output = generate_risk_report(person["name"], person["country"], age, formatted_results)
# print(output)

c:\Users\nashar\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
resume = input("jkasgfdjl")

In [66]:
import json 

with open('person.json','r') as file:
    person_data = json.load(file)
try:
    with open('output.JSON','r') as file:
        try:
            output_data = json.load(file)
        except:
            output_data = {} 
except: 
    output_data={}

for person_number, data in person_data.items():
    person = {
    "name": data["Name"],
    "country": data["Country"],
    "dob": data["DOB"],  # ISO or natural language (e.g. '20 April 1987')
    }
    age = calculate_age_from_dob(person["dob"]) 

    country_name = get_country_name(person["country"])

    query = f"{person['name']} Udaipur, {country_name} {age} years old news"
    results = search_google_cse(query,40)
    formatted_results = format_results(results)

    output = generate_risk_report(person["name"], country_name, age, formatted_results)
    print(query)
    print(output)
    test = output
    test = test[7:]
    test = test[:-3]
    test = test.replace('\n','')
    test = json.loads(test)
    output_data[person_number] = test

with open('output.JSON','w') as file:
    json.dump(output_data,file)

Vijay Malaya Udaipur, India  years old news
```json
{
  "Person Name": "Vijay Mallya",
  "Country ISO code": "IND",
  "Country Name": "India",
  "Risk Score": 10,
  "Summary": "Fugitive Vijay Mallya faces extreme risk due to major financial fraud, money laundering charges, and bank loan defaults in India. He fled to the UK and is subject to ongoing extradition attempts. Mallya was also held in contempt of court for illicit fund transfers. Billions in assets have been seized.",
  "Sources": [
    "https://www.ndtv.com/india-news/supreme-court-reserves-order-on-vijay-mallyas-plea-seeking-review-of-2017-contempt-case-verdict-2286081",
    "https://www.ndtv.com/india-news/sc-reserves-order-on-punishment-in-contempt-case-against-vijay-mallya-2815093",
    "http://www.msn.com/en-in/news/other/they-just-lock-you-up-throw-away-key-vijay-mallya-says-hes-willing-to-return-to-india-if/ar-AA1Gbl2v?apiversion=v2&noservercache=1&domshim=1&renderwebcomponents=1&wcseo=1&batchservertelemetry=1&noserver

In [30]:
test['Person Name']

'Vijay Mallya'

In [24]:
string = ''
for items in test['Sources']:
    if string == '':
        string = str(items)
    else:
        string = string + " , " + str(items)

In [18]:
code = 'TJWdzXPfjKwYpma10UdIPuJrhM_pTG1g'

In [19]:
import requests

url = "https://create.demo.sas.com/SASLogon/oauth/token"

payload = f'grant_type=authorization_code&code={code}'

headers = {
 'Accept': 'application/json',
 'Content-Type': 'application/x-www-form-urlencoded',
 'Authorization': 'Basic c2FzLmNsaTo='
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

import json 
token_dictionary = json.loads(response.text)
access_token = token_dictionary['access_token']

{"access_token":"eyJqa3UiOiJodHRwczovL2xvY2FsaG9zdC9TQVNMb2dvbi90b2tlbl9rZXlzIiwia2lkIjoibGVnYWN5LXRva2VuLWtleSIsInR5cCI6IkpXVCIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJiNTE0NTg4OS04YWIxLTQyZTMtOGNiNy1mNTNmZjRkNWU0NDEiLCJzZXNzaW9uX3NpZyI6ImI5ODJkMWQyLWFkODItNDMzOC1hZjg5LTcyZmZiZGQ5ZDRiMCIsInVzZXJfbmFtZSI6Ik5hbWFuLlNoYXJtYS5FeHRlcm5hbEBzYXMuY29tIiwib3JpZ2luIjoiYXp1cmUiLCJpc3MiOiJodHRwOi8vbG9jYWxob3N0L1NBU0xvZ29uL29hdXRoL3Rva2VuIiwiYXV0aG9yaXRpZXMiOlsiU0FTU2NvcmVVc2VycyIsIkRhdGFCdWlsZGVycyIsIkdsb3NzYXJ5Lkdsb3NzYXJ5QWRtaW5pc3RyYXRvcnMiLCJDSVMgVml5YSBBbHdheXMgT24iLCJDYXRhbG9nLlN1YmplY3RNYXR0ZXJFeHBlcnRzIiwiQXBwbGljYXRpb25BZG1pbmlzdHJhdG9ycyIsIklDVXMiLCJNaWdyYXRpb25BZG1pbnMiLCJFc3JpVXNlcnMiLCJDQVNIb3N0QWNjb3VudFJlcXVpcmVkIl0sImNsaWVudF9pZCI6InNhcy5jbGkiLCJhdWQiOlsic2FzLXRyYW5zZmVyIiwic2FzLWZvbnRzIiwic2FzLXJlcG9ydC1wYWNrYWdlcyIsInNhcy1jb21wdXRlIiwib3BlbmlkIiwic2FzLWF1ZGl0Iiwic2FzLmNsaSIsInNhcy1iYXRjaCIsInNhcy1zY2hlZHVsZXIiLCJzYXMtbGF1bmNoZXIiLCJzYXMtaWRlbnRpdGllcyIsInVhYSIsInNhcy1mb2xkZXJzIiwic2FzLWN

In [4]:
access_token

'eyJqa3UiOiJodHRwczovL2xvY2FsaG9zdC9TQVNMb2dvbi90b2tlbl9rZXlzIiwia2lkIjoibGVnYWN5LXRva2VuLWtleSIsInR5cCI6IkpXVCIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJiNTE0NTg4OS04YWIxLTQyZTMtOGNiNy1mNTNmZjRkNWU0NDEiLCJzZXNzaW9uX3NpZyI6ImI5ODJkMWQyLWFkODItNDMzOC1hZjg5LTcyZmZiZGQ5ZDRiMCIsInVzZXJfbmFtZSI6Ik5hbWFuLlNoYXJtYS5FeHRlcm5hbEBzYXMuY29tIiwib3JpZ2luIjoiYXp1cmUiLCJpc3MiOiJodHRwOi8vbG9jYWxob3N0L1NBU0xvZ29uL29hdXRoL3Rva2VuIiwiYXV0aG9yaXRpZXMiOlsiU0FTU2NvcmVVc2VycyIsIkRhdGFCdWlsZGVycyIsIkdsb3NzYXJ5Lkdsb3NzYXJ5QWRtaW5pc3RyYXRvcnMiLCJDSVMgVml5YSBBbHdheXMgT24iLCJDYXRhbG9nLlN1YmplY3RNYXR0ZXJFeHBlcnRzIiwiQXBwbGljYXRpb25BZG1pbmlzdHJhdG9ycyIsIklDVXMiLCJNaWdyYXRpb25BZG1pbnMiLCJFc3JpVXNlcnMiLCJDQVNIb3N0QWNjb3VudFJlcXVpcmVkIl0sImNsaWVudF9pZCI6InNhcy5jbGkiLCJhdWQiOlsic2FzLXRyYW5zZmVyIiwic2FzLWZvbnRzIiwic2FzLXJlcG9ydC1wYWNrYWdlcyIsInNhcy1jb21wdXRlIiwib3BlbmlkIiwic2FzLWF1ZGl0Iiwic2FzLmNsaSIsInNhcy1iYXRjaCIsInNhcy1zY2hlZHVsZXIiLCJzYXMtbGF1bmNoZXIiLCJzYXMtaWRlbnRpdGllcyIsInVhYSIsInNhcy1mb2xkZXJzIiwic2FzLWNvbm5lY3QiLCJzYXM

In [5]:
import requests
import json

url = "https://create.demo.sas.com/microanalyticScore/modules/gemini_final_trial/steps/execute"
payload = json.dumps({
  "version": 2.1,
  "inputs": [
    {
      "name": "Input_Country_",
      "value": "IN"
    },
    {
      "name": "Input_Dob_",
      "value": "10/10/2003"
    },
    {
      "name": "Input_Name_",
      "value": "Naman Sharma"
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/vnd.sas.microanalytic.module.step.output+json',
  'Authorization': f'Bearer {access_token}',
  'Cookie': 'sas-ingress-nginx=1ff4206e0fe166a82be0cbe7cbb82dee|c77aaefb6d6a3105579e88f5ea518f2b'
}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)
print(response.status_code)

{"errorCode":-1958743651,"message":"Module: gemini_final_trial, Revision: 1, Method: execute, SQLState: HY000, Severity: 4, Line 222: The DS2 PYMAS package encountered a failure in the 'execute' method.  Module: \"Gemini_Final_gemini_code_file_1_0_638924676\", Revision: 1, Method: execute, Message: Traceback (most recent call last):\n  File \"/opt/sas/viya/home/SASFoundation/misc/embscoreeng/mas2py.py\", line 1284, in invoke\n    out = up[1].get(func)[0](up[0])(*args)\n          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File \"/tmp/tmpnb61liwl/\"Gemini_Final_gemini_code_file_1_0_638924676\".py\", line 25, in execute\n    import google.generativeai as genai\nModuleNotFoundError: No module named 'google.generativeai'\n","details":["path: /microanalyticScore/modules/gemini_final_trial/steps/execute"],"errors":[],"links":[],"version":2,"moduleId":"gemini_final_trial","stepId":"execute","timestamp":2068260610,"severity":"ERROR","loggerName":"App.tk.MAS","threadId":9,"sqlState":"HY000","httpStatus

In [68]:
response_json = response.json()

In [69]:
response_json


{'links': [],
 'version': 2,
 'moduleId': 'gemini_test',
 'stepId': 'execute',
 'executionState': 'completed',
 'metadata': {'module_id': 'gemini_test', 'step_id': 'execute'},
 'outputs': [{'name': 'Output_Country_ISO', 'value': None},
  {'name': 'Output_Country_Name', 'value': None},
  {'name': 'Output_Name', 'value': None},
  {'name': 'Output_Score', 'value': None},
  {'name': 'Output_Source', 'value': None},
  {'name': 'Output_Summary', 'value': None}]}

In [20]:
access_token
import requests

url = "https://create.demo.sas.com/microanalyticScore/modules/gemini_3/steps/execute"
payload = {
  "version": 2.1,
  "inputs": [
    {
      "name": "Input_Country_",
      "value": "IN"
    },
    {
      "name": "Input_Dob_",
      "value": "10/10/2003"
    },
    {
      "name": "Input_Name_",
      "value": "Naman Sharma"
    }
  ]
}
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/vnd.sas.microanalytic.module.step.output+json',
  'Authorization': f'Bearer {access_token}',
}
response = requests.request("POST", url, headers=headers, json=payload)
print(response.text)
print(response.status_code)

{"errorCode":-1958743651,"message":"Module: gemini_3, Revision: 1, Method: execute, SQLState: HY000, Severity: 4, Line 213: The DS2 PYMAS package encountered a failure in the 'execute' method.  Module: \"Gemini_Final_gemini_code_file_1_0_1046240933\", Revision: 1, Method: execute, Message: Traceback (most recent call last):\n  File \"/opt/sas/viya/home/SASFoundation/misc/embscoreeng/mas2py.py\", line 1284, in invoke\n    out = up[1].get(func)[0](up[0])(*args)\n          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File \"/tmp/tmpnb61liwl/\"Gemini_Final_gemini_code_file_1_0_1046240933\".py\", line 13, in execute\n    from google import generativeai as genai\nImportError: cannot import name 'generativeai' from 'google' (unknown location)\n","details":["path: /microanalyticScore/modules/gemini_3/steps/execute"],"errors":[],"links":[],"version":2,"moduleId":"gemini_3","stepId":"execute","timestamp":2068265119,"severity":"ERROR","loggerName":"App.tk.MAS","threadId":9,"sqlState":"HY000","httpStatusCo

In [12]:
from google import generativeai as genai
from dotenv import load_dotenv 
import os

load_dotenv()

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel(model_name="gemini-2.5-flash")

In [14]:
prompt = """What is AI?"""

response = model.generate_content(prompt)
print(response.text)

Artificial Intelligence (AI) is a broad field of computer science dedicated to creating machines that can perform tasks that typically require human intelligence.

At its core, AI aims to enable machines to **simulate cognitive functions** like:

1.  **Learning:** Acquiring information and rules for using the information. This often involves Machine Learning (ML), where systems learn from data without explicit programming, and Deep Learning (DL), which uses neural networks inspired by the human brain.
2.  **Reasoning:** Using rules to reach approximate or definite conclusions.
3.  **Problem-Solving:** Identifying and solving problems.
4.  **Perception:** Using sensory input (like images, sound) to understand the world (e.g., Computer Vision, Speech Recognition).
5.  **Language Understanding and Generation:** Processing and producing human language (Natural Language Processing/NLP).

**Key Concepts and Types of AI:**

*   **Narrow AI (Weak AI):** This is the AI we have today. It is desi

In [ ]:
import os 
import warnings
from dotenv import load_dotenv
import datetime
from dateutil import parser
import requests 
warnings.filterwarnings('ignore')

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH_ENGINE_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")



def calculate_age_from_dob(dob_str):
    if dob_str == 'Unknown':
        return ''
    if len(dob_str) == 2:
        return int(dob_str)
    dob = parser.parse(dob_str)
    today = datetime.date.today()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    return age


def search_google_cse(query, num_results):
    url = "https://www.googleapis.com/customsearch/v1"
    results = []

    for start_index in range(1, num_results + 1, 10):
        params = {
            "key": GOOGLE_API_KEY,
            "cx": SEARCH_ENGINE_ID,
            "q": query,
            "start": start_index,
            "num": 10
        }
        response = requests.get(url, params=params)
        data = response.json()

        items = data.get("items", [])
        if not items:
            break  # no more results or quota exhausted
        results.extend(items)

    return results[:num_results]


def format_results(items):
    formatted = ""
    for i, item in enumerate(items[:40], 1):  # Top 40 results
        title = item.get("title", "")
        snippet = item.get("snippet", "")
        link = item.get("link", "")
        formatted += f"{i}. {title}\n{snippet}\n{link}\n\n"
    return formatted

def get_country_name(country_iso_code):
    prompt = f'Answer the country name only. What is the country name with ISO code - {country_iso_code}'
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?key={GEMINI_API_KEY}"

    payload = {
        "contents": [
            {
                "parts": [
                    {
                            "text": prompt
                    }
                ]
            }
        ]
    }

    header = {"Content-Type":"application/json"}
    response = requests.post(url=url,headers=header,json=payload)

    country_name = response.json()
    country_name = country_name["candidates"][0]
    country_name = country_name["content"]
    country_name = country_name["parts"][0]
    country_name = country_name["text"]
    return country_name

def generate_risk_report(name, country, age, search_text):
    prompt = f"""
        You are an analyst for credit and national security risk.

        Search results for: {name}, {country}, {age} years old:{search_text}

        Try to look for some with crime, legal issues, terrorist and financial fraud. 
        Filter the search by using the sources only from news websites; do not consider any other webpage(social media or other sources).
        If the person is a politians then risk score is 100 as they are special people with special privileges.

        Based on these search results, return a JSON with:
        1. "risk_score": integer from 0 to 10 (0 = clean, 10 = extreme risk)
        2. "summary": max 50 words on why the score was given

        The output json should have the following: -
        1. Person Name
        2. Country ISO code: 
        3. Country Name:
        4. Risk Score 
        5. Summary
        6. Sources

        dont use "" this anywhere in the summary
    """
    
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?key={GEMINI_API_KEY}"
    payload = {
        "contents": [
            {
                "parts": [
                    {
                            "text": prompt
                    }
                ]
            }
        ]
    }
    header = {
        "Content-Type":"application/json"
    }
    response = requests.post(url=url,headers=header,json=payload)
    person_data = response.json()
    person_data = person_data["candidates"][0]
    person_data = person_data["content"]
    person_data = person_data["parts"][0]
    person_data = person_data["text"]
    return person_data

# with open('person.json','r') as file:
#     person_data = json.load(file)

person = {
    "name": "Naman Sharma",
    "country": "IN",
    "dob": "10/10/2003",  # ISO or natural language (e.g. '20 April 1987')
}

age = calculate_age_from_dob(person["dob"])
country_name =get_country_name(person["country"]) 

query = f"{person['name']} {country_name} {age} years old"
results = search_google_cse(query,40)
formatted_results = format_results(results)

output = generate_risk_report(person["name"], country_name, age, formatted_results)
print("The offical output is :-\n")
print(output)
test = output
test = test[7:]
test = test[:-3]
test = test.replace('\n','')
test = json.loads(test)
print("The cleaned output is :-\n")
print(test)

The offical output is :-

```json
{
  "Person Name": "Naman Sharma",
  "Country ISO code": "IND",
  "Country Name": "India",
  "Risk Score": 0,
  "Summary": "No adverse information found in news sources. The only relevant news mention of a Naman Sharma describes him as a volunteer during the COVID-19 pandemic, indicating no credit or national security risk.",
  "Sources": [
    "https://widerimage.reuters.com/story/reuters-wins-pulitzer-for-intimate-devastating-images-of-indias-pandemic"
  ]
}
```
The cleaned output is :-

{'Person Name': 'Naman Sharma', 'Country ISO code': 'IND', 'Country Name': 'India', 'Risk Score': 0, 'Summary': 'No adverse information found in news sources. The only relevant news mention of a Naman Sharma describes him as a volunteer during the COVID-19 pandemic, indicating no credit or national security risk.', 'Sources': ['https://widerimage.reuters.com/story/reuters-wins-pulitzer-for-intimate-devastating-images-of-indias-pandemic']}


In [32]:
output['content']['parts'][0]

{'text': '```json\n{\n  "Person Name": "Naman Sharma",\n  "Country ISO code": "IND",\n  "Country Name": "India",\n  "Risk Score": 0,\n  "Summary": "No adverse media found from news websites matching the specified criteria for Naman Sharma, 21 years old. The only news article identified refers to a different individual who died in 2009.",\n  "Sources": []\n}\n```'}

In [ ]:
# test = output
#     test = test[7:]
#     test = test[:-3]
#     test = test.replace('\n','')
#     test = json.loads(test)